DWT_Dashboard
Interactive file processing and seizure detection
Claire Seibold

The default parameters are set for best results, please enter a file path and selet 'Reset Cache' for the text file to be processed. After the initial run, 'Reset Cache' can be unchecked to speed things up.


In [2]:
%env File_Folder=/Volumes/disk2s1/Expesicor/Sync/SampleFiles/0023.txt
%env Percent_Sensitivity = 10
%env Expected_Artifact_Length = 30
%env Include_Spikes = True
%env Reset_Cache = True

env: File_Folder=/Volumes/disk2s1/Expesicor/Sync/SampleFiles/0023.txt
env: Percent_Sensitivity=10
env: Expected_Artifact_Length=30
env: Include_Spikes=True
env: Reset_Cache=True


In [16]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.dates as mdates
import pywt
import os
import sys
import warnings
import time
import datetime
from itertools import islice
from ipywidgets import interact,interactive,widgets
from IPython.display import Javascript, display

In [108]:
def f(x):
    return x

SensitivityWidget = widgets.FloatSlider(
    value=10,
    min=0,
    max=20.0,
    step=0.1,
    description='Percent sensitivity: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format=':.2%',
)

interact(f,x=SensitivityWidget);

10.0

In [118]:
TimeWidget = widgets.FloatSlider(
    value=30,
    min=20,
    max=60,
    step=1,
    description='Minimum artefact length of interest: ',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

interact(f,x=TimeWidget);

40.0

In [44]:
PathWidget = widgets.Text(value = '/path/to/file/',
                          placeholder = 'Type File Path',
                          description = 'File path: ',
                          disabled = False);
def g(x):
    return x
    
interact(g,x=PathWidget)

'/Volumes/disk2s1/Expesicor/Sync/SampleFiles/0023.txt'

In [45]:
SpikeWidget = widgets.Checkbox(
              value=True,
              description='Include unusual spikes',
              disabled=False)

ResetWidget = widgets.Checkbox(
              value=False,
              description='Reset cached data',
              disabled=False)
    
interact(g,x=SpikeWidget);
interact(g,x=ResetWidget);

True

In [25]:
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))


button = widgets.Button(description="Begin Processing")
button.on_click(run_all)
display(button)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [119]:
#Functions
def ReadChunks(file,chunk_size=1024*1024*26):
    """Lazy function (generator) to read a file piece by piece.
    Default chunk size: 1024x1024"""
    while True:
        data = file.read(nChannels*chunk_size).splitlines()
        if not data:
            break
        yield data

def seekLine(file, n):
    for ignored_line in islice(file, n - 1):
        pass # skip n-1 lines

def ProcessData(data,progress):
    """Read chunks of data that comes in split by line
    Split by tab and increment the progress"""
    global X
    for item in data:
        X.append([x for x in item[:-1].split('\t')])
        progress = progress + 1
    return progress

def ProcessLine(line):
    """Fancy line processor
    Checks that no channel was cut off, appends the entries,
    Trims and returns the time"""
    global X
    empt = [1 for i in line.split('\t') if not i]
    if len(line.split('\t')) == nChannels and not empt:
        try:
            X.append([float(x) for x in line.split('\t')])
        except ValueError:
            pass

In [120]:
def GetStartTime(file):
    line = file.readline(30)
    line = line.split('\t')
    return float(line[0])

def GetTimeSinceStart(endSecond,startSecond):
    '''return the seconds since the file began'''
    return (endSecond - startSecond)/60

In [121]:
def GetChannels(file):
    """Return number of channels/columns in the file"""
    global nChannels
    init = file.readlines(1)
    nChannels = len([i.split('\t') for i in init][0])

def GetSamplingRate(file):
    init = [next(file) for x in range(2)]
    init
    t = [i.split('\t') for i in init]
    rate = round(1/(float(t[1][0]) - float(t[0][0])),0)
    return rate

In [122]:
def ProcessDWT(includeSpikes):
    """Process 256 measurements at a time with the DWT
    Take the proportion of energy contained in each coefficient
    Append result in D with number of columns = 6 x channels"""
    global D
    for i in range(0,X.shape[0]-1,DWT_blocksize):
        re = [X[0,0]] #append time
        if includeSpikes is False: upperLimit = np.percentile(X[i:i+DWT_blocksize,col],90)
        for col in range(1,nChannels):
            temp = [float(j) for j in X[i:i+DWT_blocksize,col]]
            if includeSpikes is False: temp = [0 if abs(i) > upperLimit else i for i in temp]
            decomp = pywt.wavedec(temp,w,level=DWT_frequencies)
            energy = [np.mean(decomp[l]**2) for l in range(len(decomp)-1,-1,-1)]
            re = re + [j/sum(energy) for j in energy]
        D.append(re)

def SeizureSearch(a,b,results):
    """Search blocks of length TIME_BLOCK for places of low variation
    For comparison, divide the file into 6 parts (usually 30 minute segments)"""
    col = 0
    for re in range(1,(nChannels-1)*(DWT_frequencies+1)+1):
        lastmoment = 0
        if (re-1) % (DWT_frequencies+1) == 0:
            col += 1

        #how much time is in each low variance increment: D[ts,0]-D[0,0]
        lowvar = GetLowVariance(a,b,re)

        #if more than 20 artefacts, skip
        if len(lowvar) > 0:
            for c,d in enumerate(lowvar):
                minute = D[lowvar[c],0]
                matches = [x for x in results if x[2] == col and x[0] == lowvar[c]]
                #if an artefact within one minute was found, skip
                if minute - lastmoment > b and not matches:
                    addl = [i for i,j in enumerate(results) if j[0] in range(lowvar[c]-b,lowvar[c]+b) and j[2] == col]
                    if not addl:
                        results.append([lowvar[c],re,col])
                    else:
                        results[addl[0]].append((re-1)%(DWT_frequencies+1))
                    lastmoment = minute
    return results

In [123]:
def GetLowVariance(a,b,re):
    '''Return time of blocks that have 'a' less variation than the average'''
    result = []
    ll = a*np.nanmean([np.nanstd(D[t-b:t,re]) for t in range(b,len(D[:,re]),b)])
    result.extend([t for t in range(b,len(D[:,re]),int(b/2)) if np.nanstd(D[t-b:t,re])<ll])
    return result

In [124]:
def FrequencyArtefactImage(t,path,re,col):
    '''Create image of DWT processed signal at artefact'''
    spath = path[-8:-4]
    #Get time at start and end
    min_t = max(t-300,0)
    max_t = min(t+300,len(D[:,0])-1)
    minute = (D[min_t,0] - D[0,0])/60
    maxminute = (D[max_t,0] - D[0,0])/60
    t = [datetime.datetime.utcfromtimestamp(j) for i,j in enumerate(D[min_t:max_t,0])]
    plt.title('Frequency band: '+str((re-1)%(DWT_frequencies+1)))
    plt.xlabel('Time')
    #plt.xaxis.set_major_locator(FormatStrFormatter('%.3f'))
    plt.plot(t,D[min_t:max_t,re])
    plt.show()

def OriginalArtefactImage(x,y,col,path):
    '''IN PROGRESS... Create image of DWT processed signal at artefact'''
    spath = path[-8:-4]
    t = [datetime.datetime.utcfromtimestamp(x[i]) for i,j in enumerate(x)]
    plt.title('Artefact found in channel:' + str(col))
    plt.xlabel('Time')
    plt.ylabel('V')
    myFmt = mdates.DateFormatter('%H:%M:%S')
    plt.gca().xaxis.set_major_formatter(myFmt)
    plt.plot(t,y)
    plt.gcf().autofmt_xdate()
    moment = y[int(len(y)/2)]
    plt.show()
    
def GetSeizureImage(results,b,file):
    '''IN PROGRESS... Get location of artefact in original file'''
    x = {}
    y = {}
    for i,item in enumerate(results):
        row = int(item[0])#todo: index of x?
        min_t = max(row - 500,0)#max(row*DWT_blocksize - 1000*b,0)
        max_t = min(row + 500,D.shape[0]*DWT_blocksize)#min(row*DWT_blocksize + 1000*b,D.shape[0]*DWT_blocksize)
        y[i] = [min_t,max_t,item[2]]
    
    #for i in y:
        current_line = 0
        with open(filename,'r') as file:
            #seekLine(file,jumpStart)
            for line in file:
                trimmed = line.strip('\n').split('\t')
                if float(trimmed[0]) < D[y[i][1],0] and float(trimmed[0]) > D[y[i][0],0]:
                    if i not in x.keys():
                        x[i] = [float(trimmed[0])]
                    else:
                        x[i].append(float(trimmed[0]))
                    y[i].append(float(trimmed[y[i][2]]))
                elif float(trimmed[0]) == D[y[i][1],0]:
                    OriginalArtefactImage(x[i],y[i][3:],y[i][2],filename)
                elif float(trimmed[0]) > D[y[i][1],0]:
                    break
            

In [125]:
# %%
#d1 = 128-256, d2 = 64-128, d3 = 32-64, d4 = 16-32, d5 = 8-16, a5 = 0-8
#Globals and Constants
global jumpStart
global w, nChannels, DWT_frequencies
global X, D

#D1 is 256-512 hertz with this sampling
DWT_blocksize = 512
w = pywt.Wavelet('db4')
DWT_frequencies = pywt.dwt_max_level(DWT_blocksize,w.dec_len)-1

class BreakIt(Exception): pass

In [126]:
#Initialize paths and widget information
filename = os.getenv('File_Folder', os.path.expanduser('~/kal/'))
#varianceParameter = float(os.getenv('Percent_Sensitivity'))/100
#timeParameter = np.float(os.getenv('Expected_Artifact_Length'))/2
filename = PathWidget.value
varianceParameter = SensitivityWidget.value/100
timeParameter = TimeWidget.value/2;
filename = PathWidget.value;
Include_Spikes = SpikeWidget.value;
Reset_Cache = ResetWidget.value;


#Run through entire file at first entrance
if Reset_Cache == True:
    with open(filename,'r') as file:
        #Initialize
        first_sec = GetStartTime(file)
        
        #Output desciption of file
        total = minutesSinceStart
        sys.stdout.write('\n\nFile ' + str(filename) +'\n')
        sys.stdout.write(str(nChannels-1) + ' channels exist over a length of {0:0.2f}'.format(total) + ' minutes \n\n')

        #seekLine(file,jumpStart) #start at 8:08am
        GetChannels(file)
        samplingRate = int(GetSamplingRate(file))
        if samplingRate < 8000:
             timeParameter = timeParameter * 3
        X = []
        D = []
        results = []
        progress = 0

        for data in ReadChunks(file):
            time.sleep(1)
            try:
                #Split each line and read into Matrix X
                for index,line in enumerate(data):
                    if index % 2 == 1: continue #Use every other measurement?
                    ProcessLine(line)
                    minutesSinceStart = GetTimeSinceStart(X[-1][0],first_sec)
                    if progress % (DWT_blocksize*4) == 0: #process DWT
                        X = np.asmatrix(X)
                        ProcessDWT(Include_Spikes)
                        del X
                        X = []

                    elif progress % (30 * 60 * samplingRate/2) == 0:#look for seizures, reset D every 30 minutes
                        D = np.asmatrix(D)
                        standardTimeParameter = int((samplingRate*timeParameter)/(DWT_blocksize*4))
                        results = SeizureSearch(varianceParameter,standardTimeParameter,results)
                        for i,item in enumerate(results):
                            timestamp = datetime.datetime.utcfromtimestamp(D[item[0],0])
                            sys.stdout.write('\tChannel: ' + str(item[2]))
                            sys.stdout.write('\tTimestamp: {:d}:{:02d}'.format(timestamp.hour,timestamp.minute) + '\t')
                            sys.stdout.write('\tFreq D: ' + ",".join([str((item[1]-1)%(DWT_frequencies+1)),str(item[3:])]))
                            sys.stdout.write('\n')
                        #for arte in results:
                            #print('\n Artefact at minute: %.3f' % float(GetTimeSinceStart(D[arte[0],0],first_sec)),' Mean low frequency...')
                            #print('Before artefact:', np.nanmean(D[:arte[0],arte[1]]))
                            #print('After artefact:', np.nanmean(D[arte[0]:,arte[1]]))
                        del D
                        D = []

                    progress += 1
                    if progress % 10000 == 0:
                        sys.stdout.write("\r%.3f" % float(round(minutesSinceStart,3)) + ' Minutes Processed')
                        sys.stdout.flush()

            #catch any purposeful breaks
            except BreakIt:
                print('\nForced Break')
                reset = False
                break



File /Volumes/disk2s1/Expesicor/Sync/SampleFiles/0023.txt
2 channels exist over a length of 118.72 minutes 

30.000 Minutes Processed	Channel: 1	Timestamp: 9:56		Freq D: 0,[]
	Channel: 1	Timestamp: 9:57		Freq D: 0,[]
	Channel: 1	Timestamp: 9:58		Freq D: 0,[]
	Channel: 1	Timestamp: 9:58		Freq D: 0,[]
	Channel: 1	Timestamp: 10:01		Freq D: 0,[]
	Channel: 1	Timestamp: 10:01		Freq D: 0,[1]
	Channel: 1	Timestamp: 10:02		Freq D: 0,[1]
	Channel: 1	Timestamp: 10:03		Freq D: 0,[]
	Channel: 1	Timestamp: 10:04		Freq D: 0,[]
	Channel: 1	Timestamp: 10:05		Freq D: 0,[]
	Channel: 1	Timestamp: 10:06		Freq D: 0,[]
	Channel: 1	Timestamp: 10:06		Freq D: 0,[]
	Channel: 1	Timestamp: 10:07		Freq D: 0,[]
	Channel: 1	Timestamp: 10:08		Freq D: 0,[]
	Channel: 2	Timestamp: 9:42		Freq D: 0,[1, 2]
	Channel: 2	Timestamp: 9:43		Freq D: 0,[]
	Channel: 2	Timestamp: 9:44		Freq D: 0,[]
60.000 Minutes Processed	Channel: 1	Timestamp: 10:26		Freq D: 0,[0, 0]
	Channel: 1	Timestamp: 10:27		Freq D: 0,[0]
	Channel: 1	Timestam

KeyboardInterrupt: 

In [ ]:
if len(results)>0 and len(results) < 6:
    GetSeizureImage(results, standardTimeParameter, filename)

In [ ]:
sys.stdout.write('\nCompleted.')